In [21]:
from glob import glob
import subprocess

import numpy as np
import os
from glob import glob
import matplotlib.pyplot as plt
import imageio.v3 as imageio
import json
import cv2
from tqdm import tqdm
import open3d as o3d
import shutil
import mrob
import subprocess
from joblib import Parallel, delayed

from pytorch3d import io
import torch
from pytorch3d.renderer.cameras import FoVPerspectiveCameras, PerspectiveCameras
from pytorch3d.renderer.points.rasterizer import PointsRasterizer, PointsRasterizationSettings
from pytorch3d.structures import Pointclouds
from pytorch3d.utils import cameras_from_opencv_projection 
from pytorch3d.renderer import (
    PointsRenderer,
    NormWeightedCompositor
)

from scene.dataset_readers import readColmapSceneInfo
from utils.camera_utils import loadCam
from collections import namedtuple
import open3d as o3d
import matplotlib.pyplot as plt
import numpy as np
import mrob
from tqdm import tqdm
from joblib import Parallel, delayed
import os
import pickle
import cv2
import fire

from gaussian_renderer import GaussianModel

In [4]:
def compute_point_cloud_camera_fraction(R, tvec, fx, fy, cx, cy, height, width, points, build_image=False):
    camera_matrix = torch.tensor([[fx, 0, cx],
                               [0, fy, cy],
                               [0,0,1]])
    # R = [camera.R for camera in cameras]
    # tvec = [camera.T for camera in cameras]

    camera_p3d = cameras_from_opencv_projection(R = torch.tensor(np.array(R)).unsqueeze(0).float(), 
                                                tvec = torch.tensor(np.array(tvec)).unsqueeze(0).float(), 
                                                camera_matrix = camera_matrix.unsqueeze(0).float(),
                                                image_size = torch.tensor([height, 
                                                                          width]).unsqueeze(0).float())
    
    raster_settings = PointsRasterizationSettings(
                    image_size=(height, 
                                width), 
                    radius = 0.025,
                    points_per_pixel = 1 
                    )

    # Create a points rasterizer
    

    rasterizer = PointsRasterizer(cameras=camera_p3d.cuda(), raster_settings=raster_settings)
    rasterized = rasterizer(points)

    image = None
    if build_image:
        renderer = PointsRenderer(
            rasterizer=rasterizer,
            # Pass in background_color to the alpha compositor, setting the background color 
            # to the 3 item tuple, representing rgb on a scale of 0 -> 1, in this case blue
            compositor=NormWeightedCompositor()
        )

        image = renderer(points)

    fraction_set = set(torch.unique(rasterized.idx)[1:].tolist())

    return fraction_set, image

def compute_iou_2sets(set0, set1):

    intersection_indices = set0.intersection(set1)
    union_indices = set0.union(set1)
    iou = len(intersection_indices)/len(union_indices)

    return iou, list(intersection_indices), list(union_indices)

def create_point_actor(points, colors):
    """ open3d point cloud from numpy array """
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(points)
    point_cloud.colors = o3d.utility.Vector3dVector(colors)
    point_cloud.normals = o3d.utility.Vector3dVector(np.zeros_like(points))
    return point_cloud

def rotmat2qvec(R):
    Rxx, Ryx, Rzx, Rxy, Ryy, Rzy, Rxz, Ryz, Rzz = R.flat
    K = np.array([
        [Rxx - Ryy - Rzz, 0, 0, 0],
        [Ryx + Rxy, Ryy - Rxx - Rzz, 0, 0],
        [Rzx + Rxz, Rzy + Ryz, Rzz - Rxx - Ryy, 0],
        [Ryz - Rzy, Rzx - Rxz, Rxy - Ryx, Rxx + Ryy + Rzz]]) / 3.0
    eigvals, eigvecs = np.linalg.eigh(K)
    qvec = eigvecs[[3, 0, 1, 2], np.argmax(eigvals)]
    if qvec[0] < 0:
        qvec *= -1
    return qvec

def qvec2rotmat(qvec):
    return np.array([
        [1 - 2 * qvec[2]**2 - 2 * qvec[3]**2,
         2 * qvec[1] * qvec[2] - 2 * qvec[0] * qvec[3],
         2 * qvec[3] * qvec[1] + 2 * qvec[0] * qvec[2]],
        [2 * qvec[1] * qvec[2] + 2 * qvec[0] * qvec[3],
         1 - 2 * qvec[1]**2 - 2 * qvec[3]**2,
         2 * qvec[2] * qvec[3] - 2 * qvec[0] * qvec[1]],
        [2 * qvec[3] * qvec[1] - 2 * qvec[0] * qvec[2],
         2 * qvec[2] * qvec[3] + 2 * qvec[0] * qvec[1],
         1 - 2 * qvec[1]**2 - 2 * qvec[2]**2]])


In [5]:
room_name = 'office_2'

# replica_poses_basedir = '/mnt/sdb1/home/kbotashev/iros_paper/replica_dataset_1/prepared_scenes_poses_replica'

output_basedir = '/mnt/sdb1/home/kbotashev/iros_paper/replica_dataset_1/scenes'

room_output_dir = os.path.join(output_basedir, room_name)
os.makedirs(room_output_dir, exist_ok=True)

replica_poses_dir = '/mnt/sdb1/home/kbotashev/iros_paper/replica_dataset_1/scenes/office_2/office_2_rebase/office_2_rebase_raw_poses'

interpolation = 2 if '_rebase' in replica_poses_dir else 1

scene_name = replica_poses_dir.split('/')[-1]

print(scene_name)

replica_output_dir = os.path.join(room_output_dir, scene_name, scene_name+'_raw')

scene_output_dir = os.path.join(room_output_dir, scene_name)

os.makedirs(replica_output_dir, exist_ok=True)

print('### STARTED REPLICA EXTRACTOR FOR ' + scene_name)

command = ['cd /mnt/sdb1/nerf_datasets/replica/replica-gen/ &&'
            '/mnt/sdb1/home/vpyatov/anaconda3/envs/habitat/bin/python', 
            '/mnt/sdb1/nerf_datasets/replica/replica-gen/src/example.py',
            '--dataset=/mnt/sdb1/home/kbotashev/mip-nerf_projects/data/replica_dataset/replica.scene_dataset_config.json',
            '--scene='+room_name,
            '--depth_sensor',
            '--height=1080',
            '--width=1920',
            '--interpolation='+str(interpolation),
            '--poses='+replica_poses_dir,
            '--output_path='+replica_output_dir]

subprocess.run(' '.join(command), shell=True)

#---------------------------------------------------------------------------------------
print('### FINISHED REPLICA EXTRACTOR, STARTING FORMING PCs FOR ' + scene_name)
new_dataset_dir = scene_output_dir


with open(os.path.join(replica_output_dir, 'cam_params.json')) as fp:
    camera_params = json.load(fp)['camera']

with open(os.path.join(replica_output_dir, 'transforms.json')) as fp:
    transforms = json.load(fp)    

renamed_images_dir = os.path.join(new_dataset_dir, 'images')
os.makedirs(renamed_images_dir, exist_ok=True)
colmap_emulation_dir = os.path.join(new_dataset_dir, 'sparse', '0')
os.makedirs(colmap_emulation_dir, exist_ok=True)
images_txt_path = os.path.join(colmap_emulation_dir, 'images.txt')
cameras_txt_path = os.path.join(colmap_emulation_dir, 'cameras.txt')

K = np.array([[camera_params['fx'], 0, camera_params['cx']],
            [0, camera_params['fy'], camera_params['cy']],
            [0, 0, 1]])

with open(cameras_txt_path, 'w') as camfwid:
    line_elems = ['1', 'PINHOLE', str(int(camera_params['w'])), str(int(camera_params['h'])), 
                str(camera_params['fx']), str(camera_params['fy']),
                str(camera_params['cx']), str(camera_params['cy'])]
    camfwid.write(' '.join(line_elems))

keypoints_only = False
decimation_factor = 100
general_noise_intensity = 0
peak_noise_intensity = 0
peak_pose_noise = mrob.geometry.SE3(np.random.rand(6)*peak_noise_intensity)
peak_noised_cam_indices = np.arange(40,50)

poses = []
colors = []
pcs = []
sift = cv2.SIFT_create()

poses = {}
with open(images_txt_path, "w") as fwid:
    for frame_info in tqdm(transforms['frames']):
        image_path = frame_info['file_path']
        depth_path = frame_info['depth_path']

        image = imageio.imread(os.path.join(replica_output_dir, image_path))/255
        depth = cv2.imread(os.path.join(replica_output_dir, depth_path), cv2.IMREAD_ANYDEPTH)/camera_params['scale']

        cam2world = np.array(frame_info['transform_matrix'])@np.diag([1,-1,-1,1])

        image_num_str = image_path.split('image.')[-1].split('.png')[0]

        poses[image_num_str + '.png'] = cam2world.tolist()

        if general_noise_intensity > 0 and int(image_num_str) not in peak_noised_cam_indices:
            general_pose_noise = mrob.geometry.SE3(np.random.rand(6)*general_noise_intensity)
            cam2world = general_pose_noise.mul(mrob.geometry.SE3(cam2world)).T()

        if int(image_num_str) in peak_noised_cam_indices and peak_noise_intensity > 0:
            cam2world = peak_pose_noise.mul(mrob.geometry.SE3(cam2world)).T()

        world2cam = np.linalg.inv(cam2world)
        
        line_elems = [str(int(image_num_str))]
        tvec = world2cam[:-1,-1]
        rot = world2cam[:3,:3]
        line_elems += rotmat2qvec(rot).astype(str).tolist()
        line_elems += tvec.astype(str).tolist()
        line_elems.append(str(1))
        line_elems.append(image_num_str + '.png')
        fwid.write(' '.join(line_elems))
        fwid.write('\n')
        fwid.write(' '.join(['1', '2', '3']))
        fwid.write('\n')

        u = np.arange(image.shape[1])
        v = np.arange(image.shape[0])

        uu, vv = np.meshgrid(u, v)
        pixs = np.stack([uu, vv, np.ones_like(vv)],-1)

        Kinv_P = np.einsum('ij,bcj->bci', np.linalg.inv(K), pixs)
        D_Kinv_P = np.concatenate([np.multiply(np.expand_dims(depth,-1), Kinv_P), np.expand_dims(np.ones_like(depth),-1)], -1)
        pcd = np.einsum('ij,bcj->bci', cam2world, D_Kinv_P)[:,:,:-1]
        if keypoints_only:
            image_gray = cv2.imread(os.path.join(replica_output_dir, image_path), cv2.IMREAD_GRAYSCALE)
            kps, dess = sift.detectAndCompute(image_gray, None)
            keypoint_coords = np.array([keypoint.pt[::-1] for keypoint in kps]).astype(int)
            image = image[tuple(keypoint_coords.T)].tolist()
            pcd = pcd[tuple(keypoint_coords.T)].tolist()
            decimation_factor = 1
        else:
            image = image.reshape(-1,3).tolist()[::decimation_factor]
            pcd = pcd.reshape(-1,3).tolist()[::decimation_factor]
        colors += image
        pcs += pcd
        shutil.copy(os.path.join(replica_output_dir, image_path), os.path.join(renamed_images_dir, image_num_str + '.png'))

with open(os.path.join(colmap_emulation_dir, "poses_c2w_gt.json"), "w") as outfile:
    json.dump(poses, outfile, indent = 4)
result_point_actor = create_point_actor(np.array(pcs).reshape(-1, 3), np.array(colors).reshape(-1, 3))
o3d.io.write_point_cloud(os.path.join(colmap_emulation_dir, 'points3D.ply'), result_point_actor.voxel_down_sample(voxel_size=0.025))
print('### FINISHED FORMING PCs FOR ' + scene_name)

office_2_rebase_raw_poses
### STARTED REPLICA EXTRACTOR FOR office_2_rebase_raw_poses
sim_cfg.physics_config_file = data/default.physics_config.json
[16:31:41:867660]:[Metadata] AttributesManagerBase.h(380)::createFromJsonOrDefaultInternal : <Dataset>: Proposing JSON name : default.scene_dataset_config.json from original name : default| This file does not exist.
[16:31:41:867940]:[Metadata] AssetAttributesManager.cpp(123)::createObject : Asset attributes (capsule3DSolid:capsule3DSolid_hemiRings_4_cylRings_1_segments_12_halfLen_0.75_useTexCoords_false_useTangents_false) created and registered.
[16:31:41:867970]:[Metadata] AssetAttributesManager.cpp(123)::createObject : Asset attributes (capsule3DWireframe:capsule3DWireframe_hemiRings_8_cylRings_1_segments_16_halfLen_1) created and registered.
[16:31:41:867988]:[Metadata] AssetAttributesManager.cpp(123)::createObject : Asset attributes (coneSolid:coneSolid_segments_12_halfLen_1.25_rings_1_useTexCoords_false_useTangents_false_capEnd_true)

[16:31:41:874106]:[Metadata] SceneDatasetAttributesManager.cpp(310)::readDatasetJSONCell : "stages.default_attributes" set in Attributes Manager from JSON.
[16:31:41:874253]:[Metadata] AttributesManagerBase.h(352)::buildAttrSrcPathsFromJSONAndLoad : Glob path result for /mnt/sdb1/home/kbotashev/mip-nerf_projects/data/replica_dataset/*_[0-9]/habitat/replica_stage.stage_config.json : /mnt/sdb1/home/kbotashev/mip-nerf_projects/data/replica_dataset/apartment_0/habitat/replica_stage.stage_config.json
[16:31:41:874689]:[Metadata] AttributesManagerBase.h(352)::buildAttrSrcPathsFromJSONAndLoad : Glob path result for /mnt/sdb1/home/kbotashev/mip-nerf_projects/data/replica_dataset/*_[0-9]/habitat/replica_stage.stage_config.json : /mnt/sdb1/home/kbotashev/mip-nerf_projects/data/replica_dataset/apartment_1/habitat/replica_stage.stage_config.json
[16:31:41:875006]:[Metadata] AttributesManagerBase.h(352)::buildAttrSrcPathsFromJSONAndLoad : Glob path result for /mnt/sdb1/home/kbotashev/mip-nerf_proje

Renderer: Tesla V100-PCIE-16GB/PCIe/SSE2 by NVIDIA Corporation
OpenGL version: 4.6.0 NVIDIA 510.108.03
Using optional features:
    GL_ARB_vertex_array_object
    GL_ARB_ES2_compatibility
    GL_ARB_separate_shader_objects
    GL_ARB_robustness
    GL_ARB_texture_storage
    GL_ARB_invalidate_subdata
    GL_ARB_texture_storage_multisample
    GL_ARB_multi_bind
    GL_ARB_direct_state_access
    GL_ARB_get_texture_sub_image
    GL_ARB_texture_filter_anisotropic
    GL_KHR_debug
    GL_KHR_parallel_shader_compile
Using driver workarounds:
    no-forward-compatible-core-context
    nv-egl-incorrect-gl11-function-pointers
    no-layout-qualifiers-on-old-glsl
    nv-zero-context-profile-mask
    nv-implementation-color-read-format-dsa-broken
    nv-cubemap-inconsistent-compressed-image-size
    nv-cubemap-broken-full-compressed-image-query
    nv-compressed-block-size-in-bits
[16:31:42:304031]:[Metadata] MetadataMediator.cpp(279)::getSceneInstanceAttributesByName : No existing scene instanc

OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
100%|██████████| 570/570 [01:51<00:00,  5.10it/s]


[16:33:40:044449]:[Physics] PhysicsManager.cpp(54)::~PhysicsManager : Deconstructing PhysicsManager
[16:33:40:044539]:[Scene] SceneManager.h(24)::~SceneManager : Deconstructing SceneManager
[16:33:40:044547]:[Scene] SceneGraph.h(25)::~SceneGraph : Deconstructing SceneGraph
[16:33:40:046087]:[Sensor] Sensor.cpp(69)::~Sensor : Deconstructing Sensor
[16:33:40:047703]:[Sensor] Sensor.cpp(69)::~Sensor : Deconstructing Sensor
[16:33:40:047731]:[Scene] SceneGraph.h(25)::~SceneGraph : Deconstructing SceneGraph
[16:33:40:047959]:[Scene] SemanticScene.h(48)::~SemanticScene : Deconstructing SemanticScene
[16:33:40:167292]:[Gfx] Renderer.cpp(71)::~Impl : Deconstructing Renderer
[16:33:40:167357]:[Gfx] WindowlessContext.h(17)::~WindowlessContext : Deconstructing WindowlessContext
[16:33:40:175643]:[Sim] Simulator.cpp(69)::~Simulator : Deconstructing Simulator
 ========================= Performance ======================== 
 1920 x 1080, total time 111.97 s, frame time 196.433 ms (5.1 FPS)
### FINIS

  7%|▋         | 42/570 [05:47<1:33:20, 10.61s/it]Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7effe91b4940>>
Traceback (most recent call last):
  File "/mnt/sdb1/home/kbotashev/anaconda3/envs/gs102/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
  8%|▊         | 43/570 [05:59<1:36:49, 11.02s/it]

In [22]:
from pytorch3d import io
import torch
from pytorch3d.renderer.cameras import FoVPerspectiveCameras, PerspectiveCameras
from pytorch3d.renderer.points.rasterizer import PointsRasterizer, PointsRasterizationSettings
from pytorch3d.structures import Pointclouds
from pytorch3d.utils import cameras_from_opencv_projection 
from pytorch3d.renderer import (
    PointsRenderer,
    NormWeightedCompositor
)

from scene.dataset_readers import readColmapSceneInfo
from utils.camera_utils import loadCam
from collections import namedtuple
import open3d as o3d
import matplotlib.pyplot as plt
import numpy as np
import mrob
from tqdm import tqdm
from joblib import Parallel, delayed
import os
import pickle
import cv2
import shutil

from gaussian_renderer import GaussianModel

In [16]:
task_dir_path = '/mnt/sdb1/home/kbotashev/iros_paper/ibr_dataset/scenes/playroom/playroom_base'
scene_info = readColmapSceneInfo(task_dir_path, 'images_full', eval=True)

Reading camera 225/225


In [18]:
out_dir = '/mnt/sdb1/home/kbotashev/iros_paper/ibr_dataset/scenes/playroom/playroom_task/images'
for cam_info in tqdm(scene_info.test_cameras): 
    shutil.copy(cam_info.image_path, out_dir)

100%|██████████| 29/29 [00:00<00:00, 1316.18it/s]


### REPLICA REBASE

In [1]:
import numpy as np
np.random.seed(42)
import torch
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
import random
random.seed(42)

import os
import subprocess
from tqdm import tqdm
from os import makedirs
from gaussian_renderer import render_lie, render_qtvec
import torchvision
from utils.general_utils import safe_state
from argparse import ArgumentParser
from arguments import ModelParams, PipelineParams, get_combined_args, pose_estimation_params
from gaussian_renderer import GaussianModel
from utils.image_utils import psnr
from utils.loss_utils import EdgeDetection
import numpy as np
from utils.loss_utils import l1_loss, ssim, l2_loss
from copy import deepcopy
from torchmetrics.image.lpip import LearnedPerceptualImagePatchSimilarity as LPIPS
import pypose as pp

from scene.dataset_readers import readColmapSceneInfo
from utils.camera_utils import loadCam
from collections import namedtuple
from matplotlib import pyplot as plt


import torch
import math
from diff_gaussian_rasterization_lie_pp import GaussianRasterizationSettings, GaussianRasterizer
from scene.gaussian_model import GaussianModel
from utils.sh_utils import eval_sh

import pickle

from scene.dataset_readers import CameraInfo
from PIL import Image
import mrob
import open3d as o3d

from arguments import  pose_estimation_params
from utils.general_utils import get_expon_lr_func
import cv2
from glob import glob
import copy
import json

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [11]:
room_name = 'apartment_1'
experiments_base_dir = '/mnt/sdb1/home/kbotashev/iros_paper/replica_dataset_1'

ply_path = os.path.join(experiments_base_dir, 
                        'output', room_name, 
                        'point_cloud/iteration_40000/point_cloud.ply')

gaussians = GaussianModel(sh_degree=3)
gaussians.load_ply(ply_path)

In [12]:
task_dir_path = os.path.join('/mnt/sdb1/home/kbotashev/iros_paper/replica_dataset_1/scenes/', room_name, room_name + '_rebase')
scene_info = readColmapSceneInfo(task_dir_path, 'images_original', eval=False)

Reading camera 148/148


In [13]:
args = namedtuple('args', ['resolution', 'data_device'])
args = args(1, 'cuda')

pipe = namedtuple('pipe', ['convert_SHs_python', 'compute_cov3D_python', 'debug'])
pipe = pipe(False, False, False)

bg_color = [0, 0, 0]
background = torch.tensor(bg_color, dtype=torch.float32, device="cuda")
os.makedirs(os.path.join(task_dir_path, 'images'), exist_ok=True)
for id in tqdm(range(len(scene_info.train_cameras))):
    cam_info = scene_info.train_cameras[id]
    camera = loadCam(args=args, id = id, cam_info=cam_info, resolution_scale=1, camera_type='lietorch')
    image_render = render_lie(camera, gaussians, pipe, background)["render"]
    image_render = torch.clamp(image_render, 0, 1)

    norm_image = cv2.normalize(image_render.detach().cpu().numpy().transpose(1,2,0), None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)

    norm_image = norm_image.astype(np.uint8)
    # break
    cv2.imwrite(os.path.join(task_dir_path, 'images', cam_info.image_name + '.png'), cv2.cvtColor(norm_image, cv2.COLOR_RGB2BGR))
    # break

100%|██████████| 148/148 [00:26<00:00,  5.59it/s]


: 